In [1]:
# 2. Import
import os
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv
from tqdm import tqdm
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from sentence_transformers import SentenceTransformer

# 3. Load data
df = pd.read_csv('train.csv', skiprows=1, header=None)
df.columns = ['label', 'title', 'description']
df['content'] = df['title'] + ' ' + df['description']
df = df.head(120000)  

df['label'] = df['label'].astype(int)
labels = torch.tensor(df['label'].values) - 1

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# 4. Load Sentence-BERT
sbert_model = SentenceTransformer('all-MiniLM-L6-v2')  # Model nhẹ, nhanh

# 5. Create document embeddings
embedding_file = 'doc_embeddings_sbert.pt'
if os.path.exists(embedding_file):
    print("Loading existing embeddings...")
    doc_embeddings = torch.load(embedding_file)
    if not isinstance(doc_embeddings, torch.Tensor):
        print(f"Error: doc_embeddings is {type(doc_embeddings)}, expected torch.Tensor. Regenerating embeddings...")
        os.remove(embedding_file)
        sentences = df['content'].tolist()
        embeddings = sbert_model.encode(sentences, show_progress_bar=True, convert_to_tensor=True)
        doc_embeddings = embeddings.cpu()
        torch.save(doc_embeddings, embedding_file)
        print(f"Saved SBERT embeddings to {embedding_file}")
else:
    print("Generating new embeddings with Sentence-BERT...")
    sentences = df['content'].tolist()
    embeddings = sbert_model.encode(sentences, show_progress_bar=True, convert_to_tensor=True)
    doc_embeddings = embeddings.cpu()
    torch.save(doc_embeddings, embedding_file)
    print(f"Saved SBERT embeddings to {embedding_file}")

Using device: cpu
Loading existing embeddings...


In [3]:
# 6. Build vocabulary
vectorizer = CountVectorizer(stop_words='english', max_features=10000)
X_counts = vectorizer.fit_transform(df['content'])
word_vocab = vectorizer.get_feature_names_out()
word2idx = {word: idx for idx, word in enumerate(word_vocab)}

# 7. Create word embeddings (using random init)
word_embeddings = torch.randn(len(word_vocab), doc_embeddings.size(1))  # phải cùng chiều với doc_embeddings

In [4]:
# 8. Create edges
tfidf = TfidfVectorizer(vocabulary=word_vocab)
X_tfidf = tfidf.fit_transform(df['content'])

# Word-to-Document edges (w2d) & Document-to-Word edges (d2w)
row, col, edge_weight = [], [], []
doc_offset = len(word_vocab)

for doc_idx, row_data in enumerate(X_tfidf):
    non_zero_indices = row_data.nonzero()[1]
    for word_idx in non_zero_indices:
        row.append(word_idx)
        col.append(doc_idx + doc_offset)
        edge_weight.append(X_tfidf[doc_idx, word_idx])
        row.append(doc_idx + doc_offset)
        col.append(word_idx)
        edge_weight.append(X_tfidf[doc_idx, word_idx])

In [5]:
from collections import defaultdict
import math

window_size = 20

# Word-to-Word edges (w2w) using PMI
word_count = defaultdict(int)
co_occur = defaultdict(int)
total_words = 0

# Tính PMI giữa các từ
def calculate_pmi(word_count, co_occur, total_words):
    pmi_matrix = {}
    for (word1, word2), count in co_occur.items():
        # Tính xác suất đồng xuất hiện P(w1, w2)
        p_w1_w2 = count / total_words
        # Tính xác suất xuất hiện của từng từ P(w1) và P(w2)
        p_w1 = word_count[word1] / total_words
        p_w2 = word_count[word2] / total_words
        # Tính PMI
        pmi = math.log(p_w1_w2 / (p_w1 * p_w2) + 1e-8)  # +1e-8 để tránh log(0)
        pmi_matrix[(word1, word2)] = pmi
    return pmi_matrix

for text in df['content']:
    words = text.split()
    total_words += len(words)
    for i, word in enumerate(words):
        word_count[word] += 1
        for j in range(max(0, i - window_size), min(len(words), i + window_size + 1)):
            if i != j:
                co_occur[(word, words[j])] += 1

pmi_matrix = calculate_pmi(word_count, co_occur, total_words)

w2w_threshold = 0.2
for (word1, word2), pmi in pmi_matrix.items():
    if pmi > w2w_threshold:
        if word1 in word2idx and word2 in word2idx:
            row.append(word2idx[word1])  # Chuyển từ thành chỉ mục
            col.append(word2idx[word2])
            edge_weight.append(pmi)
            row.append(word2idx[word2])
            col.append(word2idx[word1])
            edge_weight.append(pmi)

print(f"Number of edges created: {len(row)}")

Number of edges created: 13480136


In [6]:
# Thêm thư viện cần thiết
import numpy as np
import torch
import faiss
from tqdm import tqdm
import gc
import os

print("Đang xây dựng cạnh Document-to-Document với phương pháp cân bằng...")

# Chuyển document embeddings sang numpy để xử lý
doc_embeddings_np = doc_embeddings.cpu().numpy()
doc_count = doc_embeddings_np.shape[0]

# Giải phóng bộ nhớ
gc.collect()

# Thông số cấu hình
initial_threshold = 0.3  # Bắt đầu với ngưỡng thấp hơn
max_threshold = 0.7      # Ngưỡng tối đa nếu cần điều chỉnh
target_edges_per_node = 5  # Mục tiêu số lượng cạnh trung bình cho mỗi node
k = min(50, doc_count)  # Số láng giềng gần nhất cần tìm

# Chuẩn hóa embedding để sử dụng dot product cho cosine similarity
norms = np.linalg.norm(doc_embeddings_np, axis=1, keepdims=True)
normalized_embeddings = doc_embeddings_np / norms
d = normalized_embeddings.shape[1]  # Số chiều của embedding

print(f"Đang xây dựng FAISS index cho {doc_count} tài liệu...")

# Khởi tạo FAISS index (Inner Product cho vector đã chuẩn hóa tương đương cosine similarity)
index = faiss.IndexFlatIP(d)
index.add(normalized_embeddings.astype('float32'))

# Thử nghiệm với một tập nhỏ để xác định ngưỡng phù hợp
print("Đang xác định ngưỡng tương đồng phù hợp...")
sample_size = min(1000, doc_count)
sample_indices = np.random.choice(doc_count, sample_size, replace=False)
sample_embeddings = normalized_embeddings[sample_indices].astype('float32')

# Tìm kiếm k láng giềng gần nhất cho mẫu
sims, neighbors = index.search(sample_embeddings, k)

# Thống kê phân bố độ tương đồng để xác định ngưỡng
all_similarities = []
for i in range(sample_size):
    # Bỏ qua phần tử đầu tiên (chính nó)
    all_similarities.extend(sims[i][1:].tolist())

all_similarities = sorted(all_similarities, reverse=True)
total_possible_edges = len(all_similarities)

# Xác định ngưỡng dựa trên mật độ mục tiêu
target_total_edges = doc_count * target_edges_per_node
target_density = min(1.0, target_total_edges / (doc_count * (doc_count - 1) / 2))
edge_index = min(int(total_possible_edges * target_density * (sample_size / doc_count)), len(all_similarities) - 1)
adaptive_threshold = max(initial_threshold, all_similarities[edge_index])
adaptive_threshold = min(adaptive_threshold, max_threshold)  # Giới hạn trên

print(f"Ngưỡng tương đồng được xác định: {adaptive_threshold:.4f}")

# Lưu cache kết quả tìm kiếm để tránh tính toán lại
cache_file = 'faiss_neighbors_cache.npz'
use_cache = False

if os.path.exists(cache_file):
    print(f"Đang tải kết quả tìm kiếm láng giềng từ cache...")
    cache = np.load(cache_file, allow_pickle=True)
    all_neighbors = cache['neighbors']
    all_sims = cache['sims']
    use_cache = True
else:
    # Thêm cạnh D2D theo batch để tiết kiệm bộ nhớ
    batch_size = 1000
    all_neighbors = []
    all_sims = []

    print(f"Đang tìm kiếm {k} láng giềng gần nhất cho mỗi tài liệu...")
    
    for i in tqdm(range(0, doc_count, batch_size)):
        end_idx = min(i + batch_size, doc_count)
        batch = normalized_embeddings[i:end_idx].astype('float32')
        
        # Tìm kiếm k láng giềng gần nhất
        batch_sims, batch_neighbors = index.search(batch, k)
        
        # Lưu kết quả
        all_neighbors.extend(batch_neighbors.tolist())
        all_sims.extend(batch_sims.tolist())
        
        # Giải phóng bộ nhớ
        del batch_sims, batch_neighbors
        gc.collect()
    
    # Lưu cache để sử dụng sau
    np.savez_compressed(cache_file, neighbors=np.array(all_neighbors), sims=np.array(all_sims))

# Xây dựng cạnh document-to-document với ngưỡng đã điều chỉnh
d2d_edges_count = 0
print(f"Đang thêm cạnh với ngưỡng {adaptive_threshold:.4f}...")

for doc_idx in tqdm(range(doc_count)):
    neighbors = all_neighbors[doc_idx]
    sims = all_sims[doc_idx]
    
    for j in range(1, len(neighbors)):  # Bỏ qua chính nó (j=0)
        neighbor_idx = neighbors[j]
        sim_value = sims[j]
        
        # Chỉ thêm cạnh cho cặp có chỉ số (i<j) để tránh trùng lặp
        # và chỉ khi độ tương đồng vượt ngưỡng
        if sim_value > adaptive_threshold and doc_idx < neighbor_idx:
            row.append(doc_idx + doc_offset)
            col.append(neighbor_idx + doc_offset)
            edge_weight.append(float(sim_value))
            
            # Thêm cạnh ngược lại (vì đồ thị vô hướng)
            row.append(neighbor_idx + doc_offset)
            col.append(doc_idx + doc_offset)
            edge_weight.append(float(sim_value))
            
            d2d_edges_count += 1

# Giải phóng bộ nhớ
del normalized_embeddings, index, doc_embeddings_np
if use_cache:
    del all_neighbors, all_sims
gc.collect()

print(f"Hoàn thành! Đã tạo {d2d_edges_count} cặp cạnh D2D")
print(f"Tổng số cạnh hiện tại: {len(row)}")
print(f"Số cạnh trung bình mỗi tài liệu: {d2d_edges_count*2/doc_count:.2f}")

Đang xây dựng cạnh Document-to-Document với phương pháp cân bằng...
Đang xây dựng FAISS index cho 120000 tài liệu...
Đang xác định ngưỡng tương đồng phù hợp...
Ngưỡng tương đồng được xác định: 0.7000
Đang tải kết quả tìm kiếm láng giềng từ cache...
Đang thêm cạnh với ngưỡng 0.7000...


100%|███████████████████████████████████████████████████████████████████████| 120000/120000 [00:06<00:00, 17412.38it/s]


Hoàn thành! Đã tạo 415647 cặp cạnh D2D
Tổng số cạnh hiện tại: 14311430
Số cạnh trung bình mỗi tài liệu: 6.93


In [ ]:
# 7. Create graph data
from sklearn.model_selection import train_test_split
import numpy as np

# Chia dữ liệu thành train, validation, test
num_docs = doc_embeddings.size(0)
indices = np.arange(num_docs)
train_idx, temp_idx = train_test_split(indices, train_size=0.7, random_state=42, stratify=df['label'])
val_idx, test_idx = train_test_split(temp_idx, train_size=0.5, random_state=42, stratify=df['label'].iloc[temp_idx])

# Tạo masks
num_words = word_embeddings.size(0)
train_mask = torch.zeros(num_words + num_docs, dtype=torch.bool)
val_mask = torch.zeros(num_words + num_docs, dtype=torch.bool)
test_mask = torch.zeros(num_words + num_docs, dtype=torch.bool)

# Chỉ các node tài liệu được sử dụng
train_mask[num_words + train_idx] = True
val_mask[num_words + val_idx] = True
test_mask[num_words + test_idx] = True

# Kiểm tra số lượng node trong mỗi tập
print(f"Train nodes: {train_mask[num_words:].sum().item()}")
print(f"Validation nodes: {val_mask[num_words:].sum().item()}")
print(f"Test nodes: {test_mask[num_words:].sum().item()}")

# Tạo labels với kích thước num_words + num_docs
labels = torch.full((num_words + num_docs,), -1, dtype=torch.long)  # -1 cho node không có nhãn
labels[num_words:num_words + num_docs] = torch.tensor(df['label'].astype(int).values) - 1  # Nhãn cho node tài liệu

# Tạo edge_index, edge_attr
edge_index = torch.tensor([row, col], dtype=torch.long)
edge_attr = torch.tensor(edge_weight, dtype=torch.float)

# Tạo đối tượng Data
data = Data(x=torch.cat([word_embeddings, doc_embeddings], dim=0),
            edge_index=edge_index,
            edge_attr=edge_attr,
            y=labels,
            train_mask=train_mask,
            val_mask=val_mask,
            test_mask=test_mask)

# 8. Define GCN model
class TextGCN(nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super(TextGCN, self).__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, out_channels)

    def forward(self, x, edge_index, edge_weight):
        x = self.conv1(x, edge_index, edge_weight=edge_weight)
        x = F.relu(x)
        x = self.conv2(x, edge_index, edge_weight=edge_weight)
        return x

# 9. Train model with early stopping
input_dim = doc_embeddings.size(1)  # Thường là 384 với SBERT Mini
word_embeddings = torch.randn(len(word_vocab), input_dim)
x = torch.cat([word_embeddings, doc_embeddings], dim=0)

data = Data(x=x,
            edge_index=edge_index,
            edge_attr=edge_attr,
            y=labels,
            train_mask=train_mask,
            val_mask=val_mask,
            test_mask=test_mask)

model = TextGCN(input_dim, 128, 4).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
# Sử dụng ignore_index=-1 để bỏ qua node không có nhãn
criterion = nn.CrossEntropyLoss(ignore_index=-1)

# Thêm early stopping
best_val_acc = 0.0
patience = 5
patience_counter = 0
best_model_state = None
max_epochs = 20

model.train()
for epoch in range(max_epochs):
    optimizer.zero_grad()
    out = model(data.x.to(device), data.edge_index.to(device), data.edge_attr.to(device))
    # Tính loss chỉ trên các node trong train_mask
    loss = criterion(out[data.train_mask], data.y[data.train_mask].to(device))
    loss.backward()
    optimizer.step()

    # Đánh giá trên tập validation
    model.eval()
    with torch.no_grad():
        logits = model(data.x.to(device), data.edge_index.to(device), data.edge_attr.to(device))
        val_preds = logits[data.val_mask].argmax(dim=1)
        val_acc = (val_preds == data.y[data.val_mask].to(device)).float().mean().item()

    model.train()
    print(f'Epoch {epoch+1}, Loss: {loss.item():.4f}, Val Accuracy: {val_acc:.4f}')

    # Early stopping
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        best_model_state = model.state_dict()
        patience_counter = 0
    else:
        patience_counter += 1
        if patience_counter >= patience:
            print(f"Early stopping at epoch {epoch+1}")
            break

# Tải mô hình tốt nhất
model.load_state_dict(best_model_state)

# 10. Evaluate on all sets
model.eval()
with torch.no_grad():
    logits = model(data.x.to(device), data.edge_index.to(device), data.edge_attr.to(device))
    train_preds = logits[data.train_mask].argmax(dim=1)
    train_acc = (train_preds == data.y[data.train_mask].to(device)).float().mean().item()
    val_preds = logits[data.val_mask].argmax(dim=1)
    val_acc = (val_preds == data.y[data.val_mask].to(device)).float().mean().item()
    test_preds = logits[data.test_mask].argmax(dim=1)
    test_acc = (test_preds == data.y[data.test_mask].to(device)).float().mean().item()

print(f'Training Accuracy: {train_acc:.4f}')
print(f'Validation Accuracy: {val_acc:.4f}')
print(f'Test Accuracy: {test_acc:.4f}')

# 11. Save model
torch.save(model.state_dict(), 'textgcn_model.pth')

# 12. Load model (nếu cần)
# model.load_state_dict(torch.load('textgcn_model.pth'))

Train nodes: 84000
Validation nodes: 18000
Test nodes: 18000
